In [ ]:
import pandas as pd
import numpy as np
import cv2

import matplotlib.pyplot as plt

In [ ]:
train_mask = pd.read_csv('Data/train_ship_segmentations_v2.csv')
train_mask.EncodedPixels = train_mask.EncodedPixels.fillna(0)
train_mask['mask_img'] = [np.zeros((768*768, 1), dtype=int) for _ in train_mask.index]



for i in range(len(train_mask)):
    if train_mask.EncodedPixels[i] != 0:
        pxl, pxl_2 = [],[]
        
        train_mask.EncodedPixels[i] = list(map(int, train_mask.EncodedPixels[i].split(" ")))


        for j in range(0,len(train_mask.EncodedPixels[i])):
            if j%2 == 0: pxl.append(train_mask.EncodedPixels[i][j])
            else: pxl_2.append(train_mask.EncodedPixels[i][j])
        
        train_mask.EncodedPixels[i] = [list(range(pxl[i], pxl[i] + pxl_2[i])) for i in range(0, len(pxl))]
        train_mask.EncodedPixels[i] = sum(train_mask.EncodedPixels[i], [])


for i in range(len(train_mask.EncodedPixels)):
    if train_mask.EncodedPixels[i] != 0: 
        mask_pxl = train_mask.EncodedPixels[i]
        for j in range(len(mask_pxl)):
            if mask_pxl[j] == 589824:
                mask_pxl.pop()
        train_mask.mask_img[i][mask_pxl,] = 255

train_mask

In [ ]:
train_data = train_mask.iloc[0:10000,[0,2]]
train_data['image'] = [np.zeros((768*768, 1), dtype=int) for _ in train_data.index]

for img in range(len(train_data)):
    train_data.image[img] = cv2.imread('/Data/train_v2/' + train_data.ImageId[img] , cv2.IMREAD_COLOR)

train_data.image = train_data.image
train_data

In [ ]:
for i in range(len(train_data)):
    data = train_data.mask_img[i]
    data = data.reshape(768,768,1)
    data = np.rot90(data,1,axes=(1,0))
    data = np.fliplr(data)
    cv2.imwrite('/Work_data/Mask/'+ str(train_data.ImageId[i]), data, params=None)


In [ ]:
for i in range(len(train_data)):
    data = train_data.image[i]
    cv2.imwrite('/Work_data/Image/'+ str(train_data.ImageId[i]), data)